# 🏆 DEEPCHECKS CRIME LA - NIVEAU 3 : PERFORMANCE DU MODÈLE

## 🎯 Objectif
Évaluer les performances des modèles de prédiction de crimes avec Deepchecks :
- Analyse des performances par classe de crime
- Détection de biais dans les prédictions
- Vérification de la calibration du modèle
- Analyse des erreurs et cas difficiles

---

## 📦 Imports et Configuration

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Ajouter src au path
sys.path.append(os.path.join(os.getcwd(), '..'))

print("✅ Imports de base effectués")

In [ ]:
# Deepchecks pour évaluation modèle
try:
    from deepchecks.tabular import Dataset
    from deepchecks.tabular.suites import model_evaluation
    from deepchecks.tabular.checks import (
        ModelErrorAnalysis, ConfusionMatrixReport,
        RocReport, CalibrationScore,
        SegmentPerformance, PredictionDrift,
        SimpleModelComparison, WeakSegmentsPerformance
    )
    DEEPCHECKS_AVAILABLE = True
    print("✅ Deepchecks importé")
except ImportError as e:
    print(f"⚠️ Deepchecks non disponible: {e}")
    DEEPCHECKS_AVAILABLE = False

# ML et métriques
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_auc_score, roc_curve
)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Configuration des chemins
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = BASE_DIR / 'data'
MODELS_DIR = BASE_DIR / 'models'
REPORTS_DIR = BASE_DIR / 'reports' / 'deepchecks'
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print("="*80)
print("🏆 DEEPCHECKS CRIME LA - NIVEAU 3 : PERFORMANCE DU MODÈLE")
print("="*80)
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📁 Base: {BASE_DIR}")
print()

---
## 📥 Chargement des Données et du Modèle

In [ ]:
def load_preprocessed_crime_data():
    """Charge et prépare les données pour évaluation"""
    print("📦 Chargement des données pour évaluation")
    print("-" * 80)
    
    # Chercher données processed
    processed_files = list((DATA_DIR / 'processed').glob('*.csv'))
    raw_files = list((DATA_DIR / 'raw').glob('*.csv'))
    
    if processed_files:
        data_file = processed_files[0]
        df = pd.read_csv(data_file)
        print(f"✅ Données processed: {data_file.name}")
    elif raw_files:
        data_file = raw_files[0]
        df = pd.read_csv(data_file)
        
        # Appliquer preprocessing si nécessaire
        try:
            from src.data.preprocessing import clean_data
            df = clean_data(df)
            print(f"✅ Données raw + preprocessing: {data_file.name}")
        except Exception as e:
            print(f"⚠️ Preprocessing échoué: {e}")
    else:
        raise FileNotFoundError("Aucun fichier de données trouvé")
    
    # Préparer pour ML (même logique que train.py)
    if 'Crime_Group' not in df.columns and 'Crm Cd Desc' in df.columns:
        try:
            from src.models.train import map_crime_group_4
            df['Crime_Group'] = df['Crm Cd Desc'].apply(map_crime_group_4)
            print("✅ Groupes de crimes créés")
        except Exception as e:
            print(f"⚠️ Mapping crimes échoué: {e}")
    
    print(f"   Shape: {df.shape}")
    
    return df

In [ ]:
def load_best_crime_model():
    """Charge le meilleur modèle disponible"""
    print("🤖 Chargement du meilleur modèle")
    print("-" * 80)
    
    # Chercher modèles dans l'ordre de préférence
    model_candidates = [
        'stacking.joblib',      # Ensemble stacking
        'voting_soft.joblib',   # Ensemble voting
        'voting_hard.joblib',   # Ensemble voting hard
        'random_forest_baseline.joblib',  # RF baseline
        'xgboost_baseline.joblib',        # XGB baseline
        'lightgbm_baseline.joblib'        # LGB baseline
    ]
    
    model = None
    model_name = None
    
    for candidate in model_candidates:
        model_path = MODELS_DIR / candidate
        if model_path.exists():
            try:
                model = joblib.load(model_path)
                model_name = candidate.replace('.joblib', '')
                print(f"✅ Modèle chargé: {candidate}")
                break
            except Exception as e:
                print(f"⚠️ Erreur chargement {candidate}: {e}")
    
    if model is None:
        print("⚠️ Aucun modèle pré-entraîné trouvé - création d'un RandomForest de démonstration")
        model = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        model_name = 'demo_random_forest'
        print("✅ Modèle de démonstration créé (sera entraîné sur les données)")
    
    return model, model_name

In [ ]:
# Charger données et modèle
df_crime = load_preprocessed_crime_data()
model, model_name = load_best_crime_model()

---
## 🔄 Préparation des Données pour Évaluation

In [ ]:
def prepare_data_for_evaluation(df, model):
    """Prépare les données pour l'évaluation du modèle"""
    print("📝 Préparation des données pour évaluation")
    print("-" * 80)
    
    # Features standard du projet
    feature_cols = ['Hour', 'Day_of_week', 'Month_num', 'LAT', 'LON']
    
    # Ajouter features optionnelles si disponibles
    optional_features = ['Vict Age', 'AREA']
    for feat in optional_features:
        if feat in df.columns:
            feature_cols.append(feat)
    
    # Label
    label_col = 'Crime_Group' if 'Crime_Group' in df.columns else 'Crm Cd Desc'
    
    # Préparer dataset ML
    df_ml = df[feature_cols + [label_col]].dropna()
    
    X = df_ml[feature_cols]
    y = df_ml[label_col]
    
    # Split (même seed que train.py)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Si le modèle n'est pas entraîné, l'entraîner
    if not hasattr(model, 'classes_') or model.classes_ is None:
        print("🚀 Entraînement du modèle sur les données...")
        
        # Imputation si nécessaire (comme dans train.py)
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy='median')
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed = imputer.transform(X_test)
        
        # Entraîner
        model.fit(X_train_imputed, y_train)
        
        # Utiliser les données imputées pour la suite
        X_train = pd.DataFrame(X_train_imputed, columns=X_train.columns, index=X_train.index)
        X_test = pd.DataFrame(X_test_imputed, columns=X_test.columns, index=X_test.index)
        
        print("✅ Modèle entraîné")
    else:
        # Imputation pour cohérence
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy='median')
        X_test = pd.DataFrame(
            imputer.fit_transform(X_test), 
            columns=X_test.columns, 
            index=X_test.index
        )
    
    print(f"✅ Données préparées:")
    print(f"   Train: {X_train.shape[0]} échantillons")
    print(f"   Test:  {X_test.shape[0]} échantillons")
    print(f"   Features: {feature_cols}")
    print(f"   Classes: {len(np.unique(y))}")
    
    return X_train, X_test, y_train, y_test, feature_cols, label_col

In [ ]:
# Préparer les données
X_train, X_test, y_train, y_test, features, label_column = prepare_data_for_evaluation(df_crime, model)

---
## 🎯 Génération des Prédictions

In [ ]:
def generate_predictions(model, X_train, X_test, y_train, y_test):
    """Génère les prédictions et probabilités"""
    print("🎯 Génération des prédictions")
    print("-" * 80)
    
    # Prédictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Probabilités (si disponible)
    y_proba_train = None
    y_proba_test = None
    
    if hasattr(model, 'predict_proba'):
        try:
            y_proba_train = model.predict_proba(X_train)
            y_proba_test = model.predict_proba(X_test)
            print("✅ Probabilités générées")
        except Exception as e:
            print(f"⚠️ Probabilités non disponibles: {e}")
    
    # Métriques de base
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted')
    
    print(f"✅ Performances de base:")
    print(f"   Train Accuracy: {train_acc:.4f}")
    print(f"   Test Accuracy:  {test_acc:.4f}")
    print(f"   Test F1-Score:  {test_f1:.4f}")
    
    return {
        'y_pred_train': y_pred_train,
        'y_pred_test': y_pred_test,
        'y_proba_train': y_proba_train,
        'y_proba_test': y_proba_test,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'test_f1': test_f1
    }

In [ ]:
# Générer les prédictions
predictions = generate_predictions(model, X_train, X_test, y_train, y_test)

---
## 🏆 NIVEAU 3 : ÉVALUATION PERFORMANCE AVEC DEEPCHECKS

### Suite d'évaluation modèle adaptée au crime

Les checks exécutés :
1. **ConfusionMatrixReport** : Matrice de confusion détaillée
2. **ModelErrorAnalysis** : Analyse des erreurs par segments
3. **RocReport** : Courbes ROC par classe
4. **CalibrationScore** : Calibration des probabilités
5. **SegmentPerformance** : Performance par sous-groupes
6. **WeakSegmentsPerformance** : Détection de segments faibles

In [ ]:
def create_deepchecks_datasets_with_predictions(X_train, X_test, y_train, y_test, predictions, features):
    """Crée les datasets Deepchecks avec prédictions"""
    print("📝 Création des datasets Deepchecks avec prédictions")
    print("-" * 80)
    
    # Préparer DataFrames avec labels et prédictions
    train_df = X_train.copy()
    train_df['__label__'] = y_train.values if hasattr(y_train, 'values') else list(y_train)
    
    test_df = X_test.copy()
    test_df['__label__'] = y_test.values if hasattr(y_test, 'values') else list(y_test)
    
    # Features catégorielles
    cat_features = []
    for col in features:
        if col in ['Day_of_week', 'Month_num', 'AREA'] or train_df[col].dtype == 'object':
            cat_features.append(col)
    
    print(f"✅ Datasets avec prédictions créés")
    print(f"   Features catégorielles: {cat_features}")
    
    if DEEPCHECKS_AVAILABLE:
        train_dataset = Dataset(
            train_df, 
            label='__label__', 
            cat_features=cat_features
        )
        test_dataset = Dataset(
            test_df, 
            label='__label__', 
            cat_features=cat_features
        )
        return train_dataset, test_dataset
    else:
        return train_df, test_df

In [ ]:
def run_model_performance_evaluation(train_ds, test_ds, model, predictions):
    """NIVEAU 3: Évaluation complète de performance du modèle"""
    print("\n" + "="*80)
    print("🏆 NIVEAU 3: ÉVALUATION PERFORMANCE DU MODÈLE")
    print("="*80)
    
    results = {}
    
    if DEEPCHECKS_AVAILABLE:
        print("\n🔍 Exécution des checks Deepchecks:")
        print("   1. Confusion Matrix Report (matrices de confusion)")
        print("   2. Model Error Analysis (analyse des erreurs)")
        print("   3. ROC Report (courbes ROC)")
        print("   4. Calibration Score (calibration probabilités)")
        print("   5. Segment Performance (performance par segments)")
        print("   6. Weak Segments Performance (segments faibles)")
        
        # Suite d'évaluation modèle
        eval_suite = model_evaluation()
        
        print("\n⏳ Exécution de l'évaluation du modèle...")
        try:
            result = eval_suite.run(train_ds, test_ds, model)
            
            # Sauvegarder le rapport
            perf_report_path = REPORTS_DIR / f'crime_model_performance_{model_name}.html'
            result.save_as_html(str(perf_report_path))
            
            print(f"✅ Rapport performance sauvegardé: {perf_report_path.name}")
            results['deepchecks_result'] = result
            
        except Exception as e:
            print(f"⚠️ Erreur suite Deepchecks: {e}")
    
    # Analyses manuelles de performance spécifiques au crime
    print("\n📊 Analyses Performance Manuelles Spécifiques Crime:")
    
    # 1. Performance par classe de crime
    y_test_vals = test_ds['__label__'] if isinstance(test_ds, pd.DataFrame) else test_ds.label
    y_pred_vals = predictions['y_pred_test']
    
    from sklearn.metrics import classification_report
    class_report = classification_report(y_test_vals, y_pred_vals, output_dict=True)
    
    print(f"\n   📋 Performance par classe:")
    for class_name, metrics in class_report.items():
        if isinstance(metrics, dict) and 'f1-score' in metrics:
            f1 = metrics['f1-score']
            support = metrics['support']
            status = "✅" if f1 > 0.7 else "⚠️" if f1 > 0.5 else "🔴"
            print(f"      {class_name}: F1={f1:.3f} Support={support} {status}")
    
    # 2. Matrice de confusion détaillée
    cm = confusion_matrix(y_test_vals, y_pred_vals)
    classes = np.unique(np.concatenate([y_test_vals, y_pred_vals]))
    
    print(f"\n   📊 Erreurs de classification principales:")
    for i, true_class in enumerate(classes):
        for j, pred_class in enumerate(classes):
            if i != j and cm[i, j] > 0:
                error_rate = cm[i, j] / cm[i, :].sum()
                if error_rate > 0.1:  # Plus de 10% d'erreur
                    print(f"      {true_class} → {pred_class}: {cm[i,j]} erreurs ({error_rate:.1%})")
    
    # 3. Analyse des probabilités (si disponible)
    if predictions['y_proba_test'] is not None:
        proba_max = np.max(predictions['y_proba_test'], axis=1)
        confidence_mean = np.mean(proba_max)
        low_confidence = (proba_max < 0.6).sum()
        
        print(f"\n   🎯 Analyse de confiance:")
        print(f"      Confiance moyenne: {confidence_mean:.3f}")
        print(f"      Prédictions peu sûres (<60%): {low_confidence} ({low_confidence/len(proba_max):.1%})")
        
        results['confidence_stats'] = {
            'mean_confidence': confidence_mean,
            'low_confidence_pct': low_confidence/len(proba_max)
        }
    
    results['classification_report'] = class_report
    results['confusion_matrix'] = cm
    results['classes'] = classes
    
    return results

In [ ]:
# Créer les datasets avec prédictions
train_dataset, test_dataset = create_deepchecks_datasets_with_predictions(
    X_train, X_test, y_train, y_test, predictions, features
)

# Exécuter l'évaluation de performance
performance_results = run_model_performance_evaluation(
    train_dataset, test_dataset, model, predictions
)

In [ ]:
# Afficher le widget interactif Deepchecks (si disponible)
if 'deepchecks_result' in performance_results:
    print("📊 Rapport interactif Deepchecks:")
    performance_results['deepchecks_result']

---
## 🔍 Analyse Spécialisée Crime LA

In [ ]:
def specialized_crime_analysis(model, X_test, y_test, predictions, features):
    """Analyses spécialisées pour la prédiction de crimes LA"""
    print("\n🔍 ANALYSES SPÉCIALISÉES CRIME LA")
    print("-" * 80)
    
    y_pred = predictions['y_pred_test']
    
    # 1. Performance par zone géographique (si AREA disponible)
    if 'AREA' in X_test.columns:
        print("\n📍 Performance par zone LAPD:")
        
        areas = X_test['AREA'].unique()[:5]  # Top 5 zones
        for area in sorted(areas):
            mask = X_test['AREA'] == area
            if mask.sum() > 10:  # Au moins 10 échantillons
                area_acc = accuracy_score(y_test[mask], y_pred[mask])
                status = "✅" if area_acc > 0.7 else "⚠️" if area_acc > 0.5 else "🔴"
                print(f"      Zone {area}: Accuracy={area_acc:.3f} ({mask.sum()} crimes) {status}")
    
    # 2. Performance par période temporelle
    if 'Hour' in X_test.columns:
        print("\n🕐 Performance par période:")
        
        # Regrouper par périodes
        X_test_copy = X_test.copy()
        X_test_copy['period'] = pd.cut(
            X_test_copy['Hour'], 
            bins=[0, 6, 12, 18, 24], 
            labels=['Nuit (0-6h)', 'Matin (6-12h)', 'Après-midi (12-18h)', 'Soirée (18-24h)'],
            include_lowest=True
        )
        
        for period in X_test_copy['period'].cat.categories:
            mask = X_test_copy['period'] == period
            if mask.sum() > 10:
                period_acc = accuracy_score(y_test[mask], y_pred[mask])
                status = "✅" if period_acc > 0.7 else "⚠️" if period_acc > 0.5 else "🔴"
                print(f"      {period}: Accuracy={period_acc:.3f} ({mask.sum()} crimes) {status}")
    
    # 3. Analyse des erreurs critiques (crimes violents mal classés)
    print("\n🚨 Analyse erreurs critiques:")
    
    # Identifier les crimes violents
    violent_crimes_mask = (y_test == 'Violent Crime') if 'Violent Crime' in y_test.values else pd.Series([False] * len(y_test))
    
    if violent_crimes_mask.sum() > 0:
        violent_recall = recall_score(y_test, y_pred, labels=['Violent Crime'], average=None)[0] if 'Violent Crime' in y_test.values else 0
        violent_precision = precision_score(y_test, y_pred, labels=['Violent Crime'], average=None)[0] if 'Violent Crime' in y_test.values else 0
        
        print(f"      Crimes violents - Recall: {violent_recall:.3f} {'✅' if violent_recall > 0.8 else '🔴 CRITIQUE'}")
        print(f"      Crimes violents - Precision: {violent_precision:.3f} {'✅' if violent_precision > 0.7 else '⚠️'}")
        
        # Crimes violents manqués
        missed_violent = violent_crimes_mask & (y_pred != 'Violent Crime')
        if missed_violent.sum() > 0:
            print(f"      ⚠️ {missed_violent.sum()} crimes violents non détectés")
    
    # 4. Feature importance (si disponible)
    if hasattr(model, 'feature_importances_'):
        print("\n📊 Importance des features:")
        
        importances = model.feature_importances_
        feature_imp = pd.DataFrame({
            'feature': features,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        for _, row in feature_imp.head().iterrows():
            print(f"      {row['feature']}: {row['importance']:.3f}")
    
    return {
        'violent_crime_performance': {
            'recall': violent_recall if 'violent_recall' in locals() else None,
            'precision': violent_precision if 'violent_precision' in locals() else None
        }
    }

In [ ]:
# Exécuter les analyses spécialisées
specialized_results = specialized_crime_analysis(model, X_test, y_test, predictions, features)

---
## 📊 Résumé Final et Recommandations

In [ ]:
print("\n" + "="*80)
print("✅ NIVEAU 3 : ÉVALUATION PERFORMANCE - TERMINÉ")
print("="*80)

print(f"\n🏆 Résumé Performance Globale:")
print(f"   Modèle évalué: {model_name}")
print(f"   Test Accuracy: {predictions['test_acc']:.4f}")
print(f"   Test F1-Score: {predictions['test_f1']:.4f}")
print(f"   Échantillons test: {len(y_test)}")

# Évaluation qualitative
if predictions['test_acc'] > 0.85:
    quality = "🏆 EXCELLENTE"
elif predictions['test_acc'] > 0.75:
    quality = "✅ BONNE"
elif predictions['test_acc'] > 0.65:
    quality = "⚠️ ACCEPTABLE"
else:
    quality = "🔴 INSUFFISANTE"

print(f"   Qualité globale: {quality}")

# Performance par classe
if 'classification_report' in performance_results:
    print(f"\n📊 Performance par classe de crime:")
    class_report = performance_results['classification_report']
    for class_name, metrics in class_report.items():
        if isinstance(metrics, dict) and 'f1-score' in metrics:
            f1 = metrics['f1-score']
            status = "✅" if f1 > 0.7 else "⚠️" if f1 > 0.5 else "🔴"
            print(f"   {class_name}: F1={f1:.3f} {status}")

# Confiance du modèle
if 'confidence_stats' in performance_results:
    conf_stats = performance_results['confidence_stats']
    print(f"\n🎯 Analyse de Confiance:")
    print(f"   Confiance moyenne: {conf_stats['mean_confidence']:.3f}")
    print(f"   Prédictions incertaines: {conf_stats['low_confidence_pct']:.1%}")

# Recommandations
print(f"\n💡 Recommandations:")

if predictions['test_acc'] > 0.8:
    print(f"   ✅ Modèle prêt pour la production")
    print(f"   📈 Considérer le déploiement via API")
    print(f"   🔄 Mettre en place monitoring continu")
else:
    print(f"   🔧 Améliorer le modèle avant production:")
    print(f"      - Essayer d'autres algorithmes (ensemble methods)")
    print(f"      - Ajouter plus de features")
    print(f"      - Optimiser les hyperparamètres")
    print(f"      - Équilibrer mieux les classes")

# Analyses spécialisées
if specialized_results['violent_crime_performance']['recall']:
    violent_recall = specialized_results['violent_crime_performance']['recall']
    if violent_recall < 0.8:
        print(f"   🚨 CRITIQUE: Améliorer détection crimes violents (recall={violent_recall:.3f})")

print(f"\n📋 Rapports générés:")
for report_file in REPORTS_DIR.glob(f'*{model_name}*.html'):
    print(f"   📄 {report_file.name}")

print(f"\n🔄 Pipeline Deepchecks complet:")
print(f"   1. ✅ NIVEAU 1: Intégrité des données")
print(f"   2. ✅ NIVEAU 2: Drift et distribution")
print(f"   3. ✅ NIVEAU 3: Performance du modèle")
print(f"   ➡️ Prêt pour la mise en production !")

print("\n" + "="*80)